In [1]:
%cd ..

/Users/ishamrashik/Desktop/machine-learning-and-ai-projects/community-mcp-session/aws_community_day/youtube-sentiment-insights


In [2]:
import mlflow
import os 

# Set AWS credentials (replace with your actual credentials)
os.environ['AWS_ACCESS_KEY_ID'] = os.getenv('AWS_ACCESS_KEY_ID')
os.environ['AWS_SECRET_ACCESS_KEY'] = os.getenv('AWS_SECRET_ACCESS_KEY')
os.environ['AWS_DEFAULT_REGION'] = os.getenv('AWS_DEFAULT_REGION')

# Then set MLflow tracking URI
mlflow.set_tracking_uri("http://3.29.137.20:5000")

In [3]:
# Set or create an experiment
mlflow.set_experiment("Exp 5 - ML Algos with HP Tuning")

<Experiment: artifact_location='s3://mlflow-artifacts-5fb77f1c/849941219442334173', creation_time=1760259501993, experiment_id='849941219442334173', last_update_time=1760259501993, lifecycle_stage='active', name='Exp 5 - ML Algos with HP Tuning', tags={}>

In [5]:
import optuna
import mlflow
import mlflow.sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [6]:
from utilities import load_data, INTERIM_DATA_PATH

In [7]:
train_df_path = os.path.join(INTERIM_DATA_PATH, "train_processed.csv")
val_df_path = os.path.join(INTERIM_DATA_PATH, "val_processed.csv")
test_df_path = os.path.join(INTERIM_DATA_PATH, "test_processed.csv")

train_df = load_data(train_df_path)
val_df = load_data(val_df_path)
test_df = load_data(test_df_path)

2025-10-12 13:00:31,031 - model_building - DEBUG - Data loaded from data/interim/train_processed.csv
2025-10-12 13:00:31,034 - model_building - DEBUG - Data loaded from data/interim/val_processed.csv
2025-10-12 13:00:31,039 - model_building - DEBUG - Data loaded from data/interim/test_processed.csv


In [8]:

ngram_range = (1, 3)  # Trigram setting
max_features = 10000  # Set max_features to 1000 for TF-IDF

# Step 2: Vectorization using TF-IDF, fit on training data only
vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)

# Step 1: Vectorize the text comments (Bag of Words)
X_train_text = vectorizer.fit_transform(train_df['clean_comment']).toarray()
X_val_text = vectorizer.transform(val_df['clean_comment']).toarray()

# Step 2: Extract numerical features
# Define the numerical feature columns to include
numerical_features = ['word_count', 'num_stop_words', 'num_chars', 'num_chars_cleaned']

X_train_numerical = train_df[numerical_features].values
X_val_numerical = val_df[numerical_features].values

# Step 3: Combine text features with numerical features
import numpy as np

X_train = np.hstack([X_train_text, X_train_numerical])
X_val = np.hstack([X_val_text, X_val_numerical])

# Target variable
y_train = train_df['category']
y_val = val_df['category']

smote = SMOTE(random_state=42)
X_train_vec, y_train = smote.fit_resample(X_train, y_train)

# Function to log results in MLflow
def log_mlflow(model_name, model, X_train, X_val, y_train, y_val):
    with mlflow.start_run():
        # Log model type
        mlflow.set_tag("mlflow.runName", f"{model_name}_SMOTE_TFIDF_Trigrams")
        mlflow.set_tag("experiment_type", "algorithm_comparison")

        # Log algorithm name as a parameter
        mlflow.log_param("algo_name", model_name)

        # Train model
        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)

        # Log accuracy
        accuracy = accuracy_score(y_val, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_val, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log the model
        mlflow.sklearn.log_model(model, f"{model_name}_model")


# Step 6: Optuna objective function for XGBoost
def objective_xgboost(trial):
    n_estimators = trial.suggest_int('n_estimators', 50, 300)
    learning_rate = trial.suggest_float('learning_rate', 1e-4, 1e-1, log=True)
    max_depth = trial.suggest_int('max_depth', 3, 10)

    model = XGBClassifier(n_estimators=n_estimators, learning_rate=learning_rate, max_depth=max_depth, random_state=42)
    return accuracy_score(y_val, model.fit(X_train_vec, y_train).predict(X_val))


# Step 7: Run Optuna for XGBoost, log the best model only
def run_optuna_experiment():
    study = optuna.create_study(direction="maximize")
    study.optimize(objective_xgboost, n_trials=30)

    # Get the best parameters and log only the best model
    best_params = study.best_params
    best_model = XGBClassifier(n_estimators=best_params['n_estimators'], learning_rate=best_params['learning_rate'], max_depth=best_params['max_depth'], random_state=42)

    # Log the best model with MLflow, passing the algo_name as "xgboost"
    log_mlflow("XGBoost", best_model, X_train_vec, X_val, y_train, y_val)

# Run the experiment for XGBoost
run_optuna_experiment()


[I 2025-10-12 13:06:48,433] A new study created in memory with name: no-name-32f398db-d76e-4c54-9cb5-9cbba5d95323
[I 2025-10-12 13:07:47,275] Trial 0 finished with value: 0.5864577504196978 and parameters: {'n_estimators': 112, 'learning_rate': 0.006581459574081046, 'max_depth': 9}. Best is trial 0 with value: 0.5864577504196978.
[I 2025-10-12 13:08:19,916] Trial 1 finished with value: 0.5243424734191382 and parameters: {'n_estimators': 83, 'learning_rate': 0.0008762546874443324, 'max_depth': 6}. Best is trial 0 with value: 0.5864577504196978.
[I 2025-10-12 13:09:28,351] Trial 2 finished with value: 0.5590374930050364 and parameters: {'n_estimators': 138, 'learning_rate': 0.00040810574016761136, 'max_depth': 8}. Best is trial 0 with value: 0.5864577504196978.
[I 2025-10-12 13:10:06,352] Trial 3 finished with value: 0.6105204252937885 and parameters: {'n_estimators': 88, 'learning_rate': 0.0230263196600886, 'max_depth': 7}. Best is trial 3 with value: 0.6105204252937885.
[I 2025-10-12 1

KeyboardInterrupt: 